In [2]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import numpy as np
import pandas as pd
import tensorflow as tf

import time
import tqdm
import matplotlib.pyplot as plt

In [3]:
import json
import os
import datetime as dt

In [4]:
pwd

'C:\\Users\\11503'

In [5]:
cd E:\DLDATA\all

E:\DLDATA\all


def f2cat(filename: str) -> str:
    return filename.split('.')[0]

In [6]:
def f2cat(filename):
    return filename.split('.')[0]

In [7]:
f2cat('asdasd.we')

'asdasd'

In [8]:
class Simplified():
    def __init__(self, input_path='E:\\DLDATA\all'):
        self.input_path = input_path

    def list_all_categories(self):
        files = os.listdir(os.path.join(self.input_path, 'train_simplified'))
        return sorted([f2cat(f) for f in files], key=str.lower)

    def read_training_csv(self, category, nrows=None, usecols=None, drawing_transform=False):
        df = pd.read_csv(os.path.join(self.input_path, 'train_simplified', category + '.csv'),
                         nrows=nrows, parse_dates=['timestamp'], usecols=usecols)
        if drawing_transform:
            df['drawing'] = df['drawing'].apply(json.loads)
        return df

In [9]:
pwd

'E:\\DLDATA\\all'

In [10]:
s = Simplified('E:\\DLDATA\\all')

In [11]:
s.input_path

'E:\\DLDATA\\all'

In [12]:
categories = s.list_all_categories()
len(categories)

340

In [13]:
type(categories)

list

In [14]:
NCSVS = 100

In [15]:
for y,cat in tqdm.tqdm(enumerate(categories)):#将每个文件的数据根据id“依次”放进100个csv中
    df = s.read_training_csv(cat,nrows=100000)
    df['y'] = y
    df['cv'] = (df.key_id // 10 ** 7) % NCSVS #???
    for k in range(NCSVS):
        filename = 'train_k{}.csv'.format(k)
        chunk = df[df.cv == k]
        chunk = chunk.drop(['key_id'],axis=1)
        if y == 0:
            chunk.to_csv(filename,index = False)
        else:
            chunk.to_csv(filename,mode='a',header = False ,index = False)#python写入csv的模式（mode），进行大数据读写时用 mode = 'a'效果会好些
            

340it [28:56,  5.29s/it]


In [16]:
for k in range(0,100):#将csv中的数据打乱
    filename = 'train_k{}.csv'.format(k)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by='rnd').drop('rnd',axis=1)
        df.to_csv(filename + '.gz',compression='gzip',index = False)
        os.remove(filename)

for k in tqdm.tqdm(range(NCSVS)):
    filename = 'train_k{}.csv'.format(k)
    if os.path.exists(filename):
        df = pd.read_csv(filename)
        df['rnd'] = np.random.rand(len(df))
        df = df.sort_values(by='rnd').drop('rnd',axis=1)
        df.to_csv(filename + '.gz',compression='gzip',index = False)
        os.remove(filename)
print(df.shape)